### Вложенные функции

In [1]:
def parent():
    print("Printing from parent()")

    def first_child():
        print("Printing from first_child()")

    def second_child():
        print("Printing from second_child()")

    second_child()
    first_child()

In [2]:
parent()

Printing from parent()
Printing from second_child()
Printing from first_child()


### Функции как возвращаемые значения

In [3]:
def parent(num):
    def first_child():
        return "Hi, I'm Elias"

    def second_child():
        return "Call me Ester"

    if num == 1:
        return first_child
    else:
        return second_child

In [4]:
fun = parent(1)
fun()

"Hi, I'm Elias"

### Декораторы

In [2]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print("Перед вызовом функции")
        func(*args, **kwargs)
        print("После вызова функции")
    return wrapper

def say_hello(a, b=1, c=None):
    print("Привет!")

say_hello = decorator(say_hello)

say_hello(1, 2, 3)

Перед вызовом функции
Привет!
После вызова функции


In [45]:
say_hello

<function __main__.decorator.<locals>.wrapper()>

### decorator(say_hello)  === @decorator

In [47]:
def decorator(func):
    def wrapper():
        print("Перед вызовом функции")
        func()
        print("После вызова функции")
    return wrapper


@decorator
def say_hello():
    print("Привет!")

say_hello()

Перед вызовом функции
Привет!
После вызова функции


### Пример

In [11]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice


@do_twice
def p():
    print('Привет')


p()

Привет
Привет


In [15]:

def do_twice(func):

    func()
    func()



@do_twice
def p():
    print('Привет')




Привет
Привет


### Пример с аргументами функции

In [4]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice


@do_twice
def p(x):
    print(x)

@do_twice
def p2():
    print(None)

@do_twice
def p3(a, b=2):
    print(a,b)


p('Привет')
p2()
p3(22)

Привет
Привет
None
None
22 2
22 2


### Возвращаемое значение

In [5]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice


@do_twice
def p(x):
    """Документация к p()"""
    print(x)
    return True


res = p('Привет')

res


Привет
Привет


True

In [9]:
dir(p)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__type_params__']

In [25]:
p.__doc__

### Идентификация

In [10]:
print.__name__

'print'

In [11]:
p, p.__name__

(<function __main__.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>,
 'wrapper_do_twice')

In [12]:
help(p)

Help on function wrapper_do_twice in module __main__:

wrapper_do_twice(*args, **kwargs)



#### Как должно быть

In [18]:
p.__doc__

In [15]:
def fun():
    """Documentation for fun()"""

In [16]:
fun.__doc__

'Documentation for fun()'

In [39]:
help(fun)

Help on function fun in module __main__:

fun()
    Documentation for fun()



#### Правильная идентификация декораторов

In [19]:
import functools


def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def p(x):
    """Документация к p()"""
    print(x)
    return True


res = p('Привет')

res


Привет
Привет


True

In [20]:
p.__doc__

'Документация к p()'

In [45]:
p

<function __main__.p(x)>

In [23]:
p.__wrapped__(1)

1


True

In [49]:
p.__wrapped__('xxxx')

xxxx


True

### Реальный пример

In [27]:
import functools
import time


def timer(func):
    """Выводим время выполнения декорируемой функции"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__}() in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [28]:
@timer
def waste_some_time(num_times):
    """docs"""
    for _ in range(num_times):
        sum([number**2 for number in range(10_000)])


waste_some_time(999)

Finished waste_some_time() in 1.1388 secs


In [29]:
waste_some_time.__doc__

In [61]:
waste_some_time.__wrapped__(99)

### Еще пример

In [31]:
def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={repr(v)}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__}() returned {repr(value)}")
        return value
    return wrapper_debug

### Декорирование в классах

In [32]:
class TimeWaster:
    
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([number**2 for number in range(self.max_num)])

In [33]:
tw = TimeWaster(1000)

Calling __init__(<__main__.TimeWaster object at 0x0000012B386F3D40>, 1000)
__init__() returned None


In [83]:
tw.waste_time(999)

Finished waste_time() in 0.1326 secs


In [34]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [35]:
tw = TimeWaster(1000)

Finished TimeWaster() in 0.0000 secs


In [36]:
tw.waste_time(999)

### Множественные декораторы

In [46]:
@timer
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

In [45]:
# fun = timer(debug(do_twice(greet)))
# fun('abc')

Calling greet('abc')
Hello abc
Hello abc
greet() returned None
Finished greet() in 0.0004 secs


In [47]:
greet('Alina')

Calling greet('Alina')
Hello Alina
Hello Alina
greet() returned None
Finished greet() in 0.0001 secs


### Декораторы с аргументами

In [109]:
def repeat(num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat


@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

greet("World")

Hello World
Hello World
Hello World
Hello World


In [111]:
@repeat()
def greet2(name):
    print(f"Hello {name}")

greet2("World")


Hello World
Hello World


In [120]:
@repeat()
def greet3(name):
    print(f"Hello {name}")

greet3("World")

Hello World
Hello World


In [84]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

In [85]:
@repeat
def greet4(name):
    print(f"Hello {name}")

greet4("World")

Hello World
Hello World


## ДЗ

In [129]:
def decorator(func):
    def wrapper(n):
        a = func(n)
        return a[::-1]
    return wrapper


@decorator
def s(n):
    return [i for i in range(1,n+1)]


s(10)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [153]:
def repeat(ind=0):
    def decorator(func):
        def wrapper(n):
            a = func(n)
            return a[ind] if len(a) >= (ind + 1) else None
        return wrapper
    return decorator


@repeat(ind=9)
def s(n):
    return [i for i in range(1,n+1)]

s(10)

10

# ДЗ

In [12]:
def power(func):
    def dec(*arr):
        ans = func(*arr)
        ans = [i * i for i in ans]
        print('Квадратный массив')
        print(ans)
    return dec

@power
def mass(*arr):
    print('Массив')
    print(arr)
    return arr

@power
def mass1(n):
    arr = [i for i in range(n)]
    return arr

a = [1,3,4]
mass(*a)

mass1(4)

Массив
(1, 3, 4)
Квадратный массив
[1, 9, 16]
Квадратный массив
[0, 1, 4, 9]


In [9]:
import functools

def format(text : str):
    def dec(func):
        @functools.wraps(func)
        def wrapp(*args, **kwargs):
            t = text.split()
            text_f = ''
            for i in t:
                if i in '.!?:;':
                    text_f += i
                else:
                    text_f += ' '
                    text_f += i
            text_f = text_f.strip()
            s = func(text_f)
            return s
        return wrapp
    return dec


@format
def revers(text):
    return text[::-1]


text = "  Khdgdnab  uua. psosjsb  psjshgsgd   ! "
revers(text)

'!dgsghsjsp bsjsosp .auu bandgdhK'

In [21]:
def text_processing1():
    with open('./Чичиков.txt', encoding='utf-8') as f:
        return f.read()


@format(make_upper=False)
def text_processing2():
    with open('./demo.txt', encoding='utf-8') as f:
        return f.read()


@format(make_upper=True)
def text_processing3(text):
    return text[::-1]


In [22]:
import re


def format(make_upper=False):
    def outer(func):
        def inner(*args, **kwargs):
            res = func(*args, **kwargs)
            res = re.sub(r'\s+', ' ', res)
            if make_upper:
                res = res.upper()
            return res
        return inner
    return outer

In [23]:
text_processing3('сфша  шла по     шоссе и сосада сушку')

'УКШУС АДАСОС И ЕССОШ ОП АЛШ АШФС'

### сделать класс-декоратор для логирования
- имени функ
- поз арг
- ключ арг
- время начала
- время заверш
- результ  
формат = "YYYY-MM-DD HH-mm-ss | LEVEL | message"

In [1]:
import logging

logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)

handler = logging.StreamHandler()
FORMAT = '%(asctime)s | %(name)s | %(levelname)-8s | %(message)s'
formatter = logging.Formatter(fmt=FORMAT)
handler.setFormatter(formatter)
logger.addHandler(handler)

In [2]:
import time

class Timer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        start_time = time.time()
        result = self.func(*args, **kwargs)
        end_time = time.time()
        logger.info(f"Функция {self.func.__name__}")
        logger.info(f"Позиционные аргументы {args}")
        logger.info(f"Ключевые аргументы {kwargs}")
        logger.info(f"Время исполнения {end_time - start_time} секунд")
        logger.info(f"Время начала {start_time}")
        logger.info(f"Время завершения {end_time}")
        logger.info(f"Результат {result}")
        return result

@Timer
def f(a, b, arr):
    ans = [(i ** a) ** b for i in arr]
    return sum(ans)

a = [i * i for i in range(10)]
f(7, 7, a)

2025-07-10 12:30:41,267 | __main__ | INFO     | Функция f
2025-07-10 12:30:41,268 | __main__ | INFO     | Позиционные аргументы (7, 7, [0, 1, 4, 9, 16, 25, 36, 49, 64, 81])
2025-07-10 12:30:41,269 | __main__ | INFO     | Ключевые аргументы {}
2025-07-10 12:30:41,270 | __main__ | INFO     | Время исполнения 0.0 секунд
2025-07-10 12:30:41,270 | __main__ | INFO     | Время начала 1752139841.2673721
2025-07-10 12:30:41,272 | __main__ | INFO     | Время завершения 1752139841.2673721
2025-07-10 12:30:41,272 | __main__ | INFO     | Результат 3279216876603445763484999599875952222582914721396968095587186187462609420505168436962970995365


3279216876603445763484999599875952222582914721396968095587186187462609420505168436962970995365

In [19]:
import time

class timer:
    
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, type, value, traceback):
        print(time.time() - self.start_time)


def foo():
    time.sleep(2)


with timer():
    foo()

2.0060362815856934


# Замыкание
- https://habr.com/ru/articles/800239/
- https://habr.com/ru/articles/781866/

In [1]:
def outers(): 
    n = 2

    def closure(): 
        return n ** 2 
    return closure


closure_foo = outers()      # Вызываем внешнюю функцию, возвращаемая функция (замыкание) присваивается переменной 
print(closure_foo)          # <function outers.<locals>.closure at 0x7f254d6fe170> 
num = closure_foo()         # Вызываем замыкание, результат присваивается переменной 
print(num)                  # 4 

# Второй вариант вызова замыкания 
print(outers()())           # 4

<function outers.<locals>.closure at 0x0000020734304400>
4
4


In [2]:
def outers(lst):

    def closure():
        return lst[0] * 2
    return closure


x = ['a']
closure_foo = outers(x)    # Вызываем внешнюю функцию, передав ей список в качестве аргумента
print(closure_foo())       # aa

x[0] = 'b'                 # меняем единственный элемент списка
print(closure_foo())       # bb

aa
bb


In [3]:
def multiplier(factor):

    def closure(x):
        return factor * x
    return closure


double = multiplier(2)
triple = multiplier(3)

print(double(5))  # 10 результат аналогичен вызову multiplier(2)(5)
print(triple(4))  # 12 результат аналогичен вызову multiplier(3)(4)

10
12


In [4]:
def modify(foo):
    return lambda x: foo(x)


"""
# результат аналогичен обычному синтаксису
def modify(foo):

    def closure(x):
        return foo(x)
    return closure
"""


to_str = modify(str)
to_str(152)              # '152'
to_bool = modify(bool)
to_bool('John Cena')     # True
to_bool('')              # False
adder = modify(lambda x: x + 1)
adder(152)               # 153

153

In [5]:
def count_calls():
    counter = 0

    def closure(print_result=False):
        nonlocal counter
        if print_result:
            return counter
        counter += 1
        return counter
    return closure


counter = count_calls()     # Вызвав функцию, получаем счетчик (замыкание)

for _ in range(5):
    counter()               # Вызываем счетчик

print(counter(True))        # Проверяем результат подсчета: 5

for _ in range(2):
    counter()

print(counter(1))           # 7

5
7
